# Creating municipality RIDA layers

Assuming that the data we received from Septima by WeTransfer (2 folders with fgb files, plus one separate fgb file) is in the folder `data/RIDA` (from Dropbox [here](https://www.dropbox.com/scl/fo/38gkcvyfex0atdbqe3hfn/h?rlkey=z9xwq2mh6ga03c301b7f66wqx&dl=0))

This script imports each of the fgb layers we need and cuts them into muncipality-sized bites; then filters merges them to create our evaluation layers.

Run from main repo folder!

In [13]:
# import packages
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
from collections import Counter
import yaml

In [3]:
# read in kommuner data and make subfolders
muni = gpd.read_file("./data/municipality-boundaries/municipality-boundaries.gpkg")

In [4]:
# create subfolders for each municipality
for _, row in muni.iterrows():
    os.makedirs(f"data/linestring/{row.kommunekode}", exist_ok=True)
    os.makedirs(f"data/point/{row.kommunekode}", exist_ok=True)
    os.makedirs(f"data/polygon/{row.kommunekode}", exist_ok=True) 

In [5]:
# dictionary of lists: 
# key is type of data (point/linestring/polygon);
# within each item, list of .fgb filenames for that data type

rida_dict = {}

rida_dict["point"] = [
    "facilit_indkoeb.fgb",
    "facilit_overnatning.fgb",
    "facilit_rasteplads.fgb",
    'facilit_service.fgb',  
    'land_besoegs.fgb',  
    "land_fortid.fgb",
    'land_landemaerke.fgb',
    'land_udflugt.fgb',
]

rida_dict["linestring"] = [
    'land_beskyttnatur_linje.fgb',
    'rute_anden.fgb',
    'rute_cykel.fgb',
    'rute_vandre.fgb',
    'vej_basislinje.fgb',
    'vej_hastighedsgraense.fgb',
    'vej_myndighed.fgb',
    'vej_type.fgb'
]

rida_dict["polygon"] = [
    'land_anvendelse.fgb',
    'land_beskyttnatur_flade.fgb',
    'land_ejerskab.fgb',
    'land_frednatpark.fgb',
    'land_vaerdifuld.fgb'
]

geomtype_dict = {
    "point": "Point",
    "linestring": "LineString",
    "polygon": "Polygon"
}

In [7]:
%%time
# read in fgb file; 
# for each of the kommuner, clip fgb file to its extent; 
# then save in corresponding folder
for k, v in rida_dict.items():

    print(f"reading {k} data")
    for fgb in v:
        # read in fgb file
        print(f"\t {fgb}...")
        name = fgb.replace(".fgb", "")

        gdf = gpd.read_file(f"./data/RIDA/{fgb}")       
        
        assert gdf.crs == muni.crs

        # for each municipality: clip, clean, and save
        for _, row in muni.iterrows():
            
            gdf_curr = gdf.copy()
            
            # clip to municipality boundary
            gdf_curr = gdf_curr.clip(row.geometry)
                
            # get the 3 columns "gruppe", "type", "geometry":
            if "gruppe" in gdf_curr.columns:
                gdf_curr = gdf_curr[["gruppe", "type", "geometry"]]
            else:
                gdf_curr = gdf_curr[["type", "geometry"]]
                gdf_curr["gruppe"] = name
                gdf_curr = gdf_curr[["gruppe", "type", "geometry"]]

            # explode
            gdf_curr = gdf_curr.explode(index_parts=False)

            # remove non-expected type geometries
            gdf_curr = gdf_curr[gdf_curr.geometry.type == geomtype_dict[k]].copy()
            gdf_curr = gdf_curr.reset_index(drop=True)

            # save
            if not gdf_curr.empty:
                gdf_curr.to_file(f"./data/{k}/{row.kommunekode}/{name}.gpkg")    
        
        del gdf

reading point data
	 facilit_indkoeb.fgb...
	 facilit_overnatning.fgb...
	 facilit_rasteplads.fgb...
	 facilit_service.fgb...
	 land_besoegs.fgb...
	 land_fortid.fgb...
	 land_landemaerke.fgb...
	 land_udflugt.fgb...
reading linestring data
	 land_beskyttnatur_linje.fgb...
	 rute_anden.fgb...
	 rute_cykel.fgb...
	 rute_vandre.fgb...
	 vej_basislinje.fgb...
	 vej_hastighedsgraense.fgb...
	 vej_myndighed.fgb...
	 vej_type.fgb...
reading polygon data
	 land_anvendelse.fgb...
	 land_beskyttnatur_flade.fgb...
	 land_ejerskab.fgb...
	 land_frednatpark.fgb...
	 land_vaerdifuld.fgb...
CPU times: user 5h 49min 26s, sys: 7min 14s, total: 5h 56min 40s
Wall time: 8h 15min 39s
